In [ ]:
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
%matplotlib inline 

import cv2
from skimage.io import imread
from skimage import color

In [ ]:
# load the two images
base = imread('ut_plazza_2.jpg')
query = imread('ut_plazza_1.jpg')

plt.figure(figsize=(18,8))

plt.subplot(121)
plt.imshow(base)
plt.axis('off')
plt.title('base image', fontsize=20)

plt.subplot(122)
plt.imshow(query)
plt.axis('off')
plt.title('query image', fontsize=20)

plt.show()

In [ ]:
# keypoints for first image
base_gray = np.uint8(color.rgb2gray(base)*255)

descriptor = cv2.SIFT_create()
# for older versions of opencv, try:
# descriptor = cv2.xfeatures2d.SIFT_create()
keypoints_base, features_base = descriptor.detectAndCompute(base_gray, None)

print('first keypoint: \n', 
      'keypoint coordinates:', keypoints_base[0].pt, '\n',
      'keypoint scale:', keypoints_base[0].size, '\n',
      'keypoint angle:', keypoints_base[0].angle
     )

print('feature vector of first keypoint: \n',features_base[0])

In [ ]:
plt.figure(figsize=(18,10))

plt.subplot(121)
plt.imshow(cv2.drawKeypoints(base_gray, keypoints_base, None, 
                             color=(255,0,0)))
plt.axis('off')
plt.title('display all keypoints found', fontsize=20)

plt.subplot(122)
keypoints_reduced = keypoints_base[0:len(keypoints_base):10]
plt.imshow(cv2.drawKeypoints(base_gray, keypoints_reduced, None, 
                             color=(255,0,0),
                             flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS))
plt.axis('off')
plt.title('indicating scale and angle', fontsize=20)

plt.show()

In [ ]:
# keypoints for second image
query_gray = np.uint8(color.rgb2gray(query)*255)
keypoints_query, features_query = descriptor.detectAndCompute(query_gray, None)

In [ ]:
# matching points based on their distance
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

matches = bf.match(features_base,features_query)
print("number of matches (Brute force):", len(matches))

# Sort the features in order of distance.
# The points with small distance (more similarity) are ordered first in the vector
sorted_matches = sorted(matches, key = lambda x:x.distance)

for k in np.arange(0,10):
    print('keypoints {} (base) and {} (query) have a feature space distance of {:.2f}'.format(
     sorted_matches[k].trainIdx, sorted_matches[k].queryIdx, sorted_matches[k].distance))

In [ ]:
# show the first 100 matches
matches_img = cv2.drawMatches(base,keypoints_base,query,keypoints_query,sorted_matches[:100],
                           None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

plt.figure(figsize=(18,10))
plt.imshow(matches_img)
plt.axis('off')
plt.title('the best hundred matches', fontsize=20)

plt.show()

In [ ]:
# find homography
# convert the keypoint coordinates to numpy arrays
kp_b = np.float32([kp.pt for kp in keypoints_base])
kp_q = np.float32([kp.pt for kp in keypoints_query])

# construct the two sets of points
pt_b = np.float32([kp_b[m.queryIdx] for m in sorted_matches])
pt_q = np.float32([kp_q[m.trainIdx] for m in sorted_matches])

# Calculate Homography between source and destination points
# 4 is the RANSAC reprojection threshold, i.e. the maximum allowed 
# distance to treat a point pair as an inlier.
h, status = cv2.findHomography(pt_q, pt_b,  cv2.RANSAC, 4)

In [ ]:
# create panorama by warping query image
width = base.shape[1] + query.shape[1]
height = base.shape[0] + query.shape[0]

result = cv2.warpPerspective(query, h, (width, height))
result[0:base.shape[0], 0:base.shape[1]] = base

plt.figure(figsize=(20,10)) 

# uncomment next line to cut to shape
#result= result[0:680,0:1580]

plt.imshow(result)
plt.axis('off')
plt.show()

## In class exercise:
copying only the necessary lines from above, panorama stitch your own image

In [ ]:
base = imread('DKU_left.jpg')

print(base.shape)
divider = 3
base = cv2.resize(base,(base.shape[1]//divider, base.shape[0]//divider))   # // is division and rounding down
print(base.shape)

In [ ]:
query = imread('DKU_right.jpg')
query = cv2.resize(query,(query.shape[1]//divider, query.shape[0]//divider))
print(query.shape)